In [31]:
# Required deps
# bs4, pandas, requests, re, lxml

In [40]:
"""
Get the list of all pokemon with anchors by pokedex number
"""

from bs4 import BeautifulSoup as bs
import requests
import re

BASE_URL = 'https://bulbapedia.bulbagarden.net'

pokemon_list_url = f'{BASE_URL}/wiki/List_of_Pok%C3%A9mon_by_National_Pok%C3%A9dex_number'
pokemon_list_data = requests.get(pokemon_list_url).text

soup = bs(pokemon_list_data)

In [41]:
"""
filter out the tables
dom structure:
  h3
    span
        "Generation X"
  table
"""

def generational_span_filter(header):
    span = header.find('span')
    if span is not None:
        matches = re.match('^Generation [IVML]+$', span.text)
        return matches is not None
    return False

generation_headers = filter(generational_span_filter, soup.find_all('h3'))

html_tables = [(header, header.findNext('table')) for header in generation_headers]

In [46]:
"""
convert to pandas data frames, export csv
"""

import pandas as pd

for t in html_tables:
    header, table = t
    table_data = pd.read_html(str(table))[0]
    images = [(
            row.find('img')['src'],
            BASE_URL + row.find('a')['href'])
        for row in table.find_all('tr')[1:]]
    # pandas is unable to parse a > img > src, retroactively put into DF
    # likewise add indivdual page url
    table_data['MS'] = [images[i][0] for i in table_data.index]
    table_data['url'] = [images[i][1] for i in table_data.index]
    table_data.to_csv(f'data/{header.text}.csv')